In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

## Hypothesis testing

- method for inferring population parameters based on sample data
- structured approach for evaluating claims or assumptions about a population using empirical evidence
- two complementary statements:
    - null hypothesis ($H_0$): statement of no effect, difference, or relationship
        - represents the status quo or the current understanding
    - alternative hypothesis ($H_1$): statement that contradicts the null hypothesis
        - represents the claim or the new understanding that the researcher wants to prove

##### Errors in hypothesis testing

|  | $H_0$ is true | $H_0$ is false |
| --- | --- | --- |
| Reject $H_0$ | Type I error | Correct decision |
| Do not reject $H_0$ | Correct decision | Type II error |

### Scipy functions

<img alt="picture 0" src="https://cdn.jsdelivr.net/gh/sharatsachin/images-cdn@master/images/736d24e80f6bcbe81df80c7e13ad06e1ac0fc5d26ffc3ce1559e3993c367e067.png" width="1200" />  

In [2]:
a = stats.t.cdf(1.156, 5)
print(f't-distribution cdf(1.156, 5) = {a:.4f}') # cumulative distribution function

a, b, c = stats.norm.cdf(0.674), stats.norm.cdf(1.96), stats.norm.cdf(2.576) 
print(f'norm-distribution cdf(0.674) = {a:.4f}, cdf(1.96) = {b:.4f}, cdf(2.576) = {c:.4f}')

a = stats.t.ppf(0.85, 5)
print(f't-distribution ppf(0.85, 5) = {a:.4f}') # point percentile function

a, b, c = stats.norm.ppf(0.75), stats.norm.ppf(0.975), stats.norm.ppf(0.995) 
print(f'norm-distribution ppf(0.75) = {a:.4f}, ppf(0.975) = {b:.4f}, ppf(0.995) = {c:.4f}')

t-distribution cdf(1.156, 5) = 0.85
norm-distribution cdf(0.674) = 0.75, cdf(1.96) = 0.98, cdf(2.576) = 1.00
t-distribution ppf(0.85, 5) = 1.16
norm-distribution ppf(0.75) = 0.67, ppf(0.975) = 1.96, ppf(0.995) = 2.58


<img alt="picture 3" src="https://cdn.jsdelivr.net/gh/sharatsachin/images-cdn@master/images/71bb8b49827870830b3f7c2af5a95d4a2806bc9f730971e02133ecd3d79023f6.png" width="1000" />  

### T-test
- used to determine whether there is a significant difference between the means of two groups or between a sample mean and a known value
- particularly useful when dealing with small sample sizes or when the population standard deviation is unknown
- assumptions:
    - in each group, the data are approximately normally distributed
    - homogeneity of variances of the two groups
    - independence of observations within each group
- types:
    - one-sample t-test: compares the mean of a single sample to a known value or population mean
    - independent two-sample t-test: compares the means of two independent groups
    - paired t-test: compares means from the same group at different times or under different conditions

#### Types of t-tests

| Info | One-sample t-test | Independent two-sample t-test | Paired t-test |
| --- | --- | --- | --- |
| Synonyms | Student’s t-test | Independent groups / samples t-test, Equal variances t-test, Pooled t-test, Unequal variances t-test | Dependent samples t-test |
| Data | one sample | two independent samples | paired samples |
| Purpose | is population mean equal to a specific value or not | are population means for two different groups equal or not | is difference between paired measurements for a population zero or not |
| Example: test if... | mean heart rate of group of people $= 65$ or not | mean HR for two groups of people are the same or not | mean difference in HR for group of people before and after exercise is zero or not |
| Estimate of population $\mu$ | sample average | sample average for each group | sample average of differences in paired measurements |
| Population $\sigma$ | unk., use sample std. dev. | unk., use sample std. devs. for each group | unk., use sample std. dev. of differences in paired measurements |
| Degrees of freedom | observations in sample $- 1$, or $n-1$ | $n_1 + n_2 - 2$ | paired observations in sample $- 1$, or $n-1$ |


In [3]:
x = stats.t.rvs(10, size=10000) # generate random numbers from t-distribution
m, v, s, k = stats.t.stats(df=10, moments='mvsk') # get mean, variance, skew, kurtosis of t-distribution with df=10 (theoretical)
n, (smin, smax), sm, sv, ss, sk = stats.describe(x) # get mean, variance, skew, kurtosis of sample (empirical)
print(f'distribution: mean = {m:.4f}, variance = {v:.4f}, skew = {s:.4f}, kurtosis = {k:.4f}')
print(f'sample: mean = {sm:.4f}, variance = {sv:.4f}, skew = {ss:.4f}, kurtosis = {sk:.4f}')

distribution: mean = 0.00, variance = 1.25, skew = 0.00, kurtosis = 1.00
sample: mean = 0.02, variance = 1.24, skew = 0.01, kurtosis = 0.81


##### Single sample t-test
- formula: $$t = \frac{\bar{x} - \mu}{s / \sqrt{n}}$$
    - $\bar{x}$: sample mean
    - $\mu$: population mean
    - $s$: sample standard deviation
    - $n$: sample size

In [4]:
# data from https://www.jmp.com/en_in/statistics-knowledge-portal/t-test/one-sample-t-test.html
sample = [20.7, 27.46, 22.15, 19.85, 21.29, 24.75, 20.75, 22.91, 25.34, 20.33, 21.54, 21.08, 22.14, 
          19.56, 21.1, 18.04, 24.12, 19.95, 19.72, 18.28, 16.26, 17.46, 20.53, 22.12, 25.06, 22.44, 
          19.08, 19.88, 21.39, 22.33, 25.79]
n = len(sample)
mu = 20
print(f'sample size = {n}, using using scipy.stats.ttest_1samp()')
# t-test for one sample
t, p = stats.ttest_1samp(sample, mu)
print(f't = {t:.4f}, p = {p:.4f}, {"accept" if p > 0.05 else "reject"} null hypothesis ')

print(f'using manual calculation')
# t-test for one sample (manual)
# 1. calculate t-value
t = (np.mean(sample) - mu) / (np.std(sample, ddof=1) / np.sqrt(n)) # ddof=1 for sample standard deviation
# 2. decide significance level
alpha = 0.05
# 3. calculate critical value
t_critical = stats.t.ppf(1-alpha/2, n-1) # 1-alpha/2 for two-tailed test
# 4. compare t-value and critical value
print(f"t = {t:.4f}, t_critical = {t_critical:.4f}, {'accept' if np.abs(t) <= t_critical else 'reject'} null hypothesis")
# 5. other method (calculate p-value)
p = stats.t.sf(np.abs(t), n-1) * 2 # two-tailed test, sf (survival function) = 1 - cdf
print(f"t = {t:.4f}, p = {p:.4f}, {'accept' if p > alpha else 'reject'} null hypothesis")

sample size = 31, using using scipy.stats.ttest_1samp()
t = 3.07, p = 0.00, reject null hypothesis 
using manual calculation
t = 3.07, t_critical = 2.04, reject null hypothesis
t = 3.07, p = 0.00, reject null hypothesis


##### Two sample t-test
- formula: $$t = \frac{\bar{x}_1 - \bar{x}_2}{s_p \sqrt{\frac{1}{n_1} + \frac{1}{n_2}}}$$
    - $\bar{x}_1, \bar{x}_2$: sample means
    - $s_p$: pooled standard deviation
    - $n_1, n_2$: sample sizes
    - pooled standard deviation: $$s_p = \sqrt{\frac{(n_1 - 1)s_1^2 + (n_2 - 1)s_2^2}{n_1 + n_2 - 2}}$$

In [5]:
# body fat percentages, data from https://www.jmp.com/en_in/statistics-knowledge-portal/t-test/two-sample-t-test.html
men = [13.3, 6.0, 20.0, 8.0, 14.0, 19.0, 18.0, 25.0, 16.0, 24.0, 15.0, 1.0, 15.0]
women = [22.0, 16.0, 21.7, 21.0, 30.0, 26.0, 12.0, 23.2, 28.0, 23.0]
n1, n2 = len(men), len(women)
x1, x2 = np.mean(men), np.mean(women)
s1, s2 = np.std(men, ddof=1), np.std(women, ddof=1)
print(f'n1 = {n1}, n2 = {n2}, x1 = {x1:.4f}, x2 = {x2:.4f}, s1 = {s1:.4f}, s2 = {s2:.4f}')

sp = np.sqrt(((n1-1)*s1**2 + (n2-1)*s2**2) / (n1+n2-2)) # pooled standard deviation
t = (x1 - x2) / (sp * np.sqrt(1/n1 + 1/n2)) # t-value
df = n1 + n2 - 2
crit = stats.t.ppf(1-alpha/2, df) # critical value
p = stats.t.sf(np.abs(t), df) * 2 # two-tailed test
print(f't = {t:.4f}, p = {p:.4f}, {"accept" if p > 0.05 else "reject"} null hypothesis')
print(f't = {t:.4f}, t_critical = {crit:.4f}, {"accept" if np.abs(t) <= crit else "reject"} null hypothesis')

n1 = 13, n2 = 10, x1 = 14.95, x2 = 22.29, s1 = 6.84, s2 = 5.32
t = -2.80, p = 0.01, reject null hypothesis
t = -2.80, t_critical = 2.08, reject null hypothesis


##### Paired t-test
- formula: $$t = \frac{\bar{d}}{s_d / \sqrt{n}}$$
    - $\bar{d}$: sample mean of differences
    - $s_d$: standard deviation of differences
    - $n$: sample size

In [6]:
# exam scores, data from https://www.jmp.com/en_in/statistics-knowledge-portal/t-test/paired-t-test.html
exam1 = [63, 65, 56, 100, 88, 83, 77, 92, 90, 84, 68, 74, 87, 64, 71, 88]
exam2 = [69, 65, 62, 91, 78, 87, 79, 88, 85, 92, 69, 81, 84, 75, 84, 82]
n = len(exam1)
diff = np.array(exam2) - np.array(exam1)
t, p = stats.ttest_1samp(diff, 0) # one-sample t-test

print(f'n = {n}, using using scipy.stats.ttest_1samp()')
print(f't = {t:.4f}, p = {p:.4f}, {"accept" if p > 0.05 else "reject"} null hypothesis ')

print(f'using manual calculation')
# t-test for paired samples (manual)
t = np.mean(diff) / (np.std(diff, ddof=1) / np.sqrt(n)) # ddof=1 for sample standard deviation
alpha = 0.05
t_critical = stats.t.ppf(1-alpha/2, n-1)
print(f"t = {t:.4f}, t_critical = {t_critical:.4f}, {'accept' if np.abs(t) <= t_critical else 'reject'} null hypothesis")
p = stats.t.sf(np.abs(t), n-1) * 2
print(f"t = {t:.4f}, p = {p:.4f}, {'accept' if p > alpha else 'reject'} null hypothesis")

n = 16, using using scipy.stats.ttest_1samp()
t = 0.75, p = 0.46, accept null hypothesis 
using manual calculation
t = 0.75, t_critical = 2.13, accept null hypothesis
t = 0.75, p = 0.46, accept null hypothesis


### Z-test
- used to determine whether there is a significant difference between the means of two groups or between a sample mean and a known value
- similar to t-test, but used when the sample size is large (typically $n > 30$) or the population standard deviation is known
- assumptions:
    - data are normally distributed
    - population standard deviation is known
    - sample size is large
- types:
    - one-sample z-test: compares the mean of a single sample to a known value or population mean
    - two-sample z-test: compares the means of two independent groups
    - proportion z-test: compares the proportion of successes in a sample to a known proportion

#### Types of z-tests

| Info | One-sample z-test | Two-sample z-test | Proportion z-test |
| --- | --- | --- | --- |
| Synonyms | - | Independent groups / samples z-test | - |
| Data | one sample | two independent samples | one sample |
| Purpose | is population mean equal to a specific value or not | are population means for two different groups equal or not | is population proportion equal to a specific value or not |
| Example: test if... | mean HR of group of people $= 65$ or not | mean HR for two groups of people are the same or not | proportion of people who like ice cream is 0.5 or not |
| Estimate of population $\mu$ | sample average | sample average for each group | sample proportion |
| Population $\sigma$ | known | known | known |
| Degrees of freedom | - | - | - |

##### Single sample z-test
- formula: $$z = \frac{\bar{x} - \mu}{\sigma / \sqrt{n}}$$
    - $\bar{x}$: sample mean
    - $\mu$: population mean
    - $\sigma$: population standard deviation
    - $n$: sample size

In [7]:
# single sample z-test, data from https://www.theopeneducator.com/doe/hypothesis-Testing-Inferential-Statistics-Analysis-of-Variance-ANOVA/Single-Sample-Z-Test
sample = [67, 73, 73, 70, 71, 68, 65, 71, 67, 73, 67, 73, 71, 67, 71, 72, 69, 70, 69, 66, 75, 73, 68, 68, 71, 65, 73, 72, 66, 66]
n = len(sample)
mu, sigma = 70, 3
z = (np.mean(sample) - mu) / (sigma / np.sqrt(n))
alpha = 0.05
z_critical = stats.norm.ppf(1-alpha/2)
p = stats.norm.sf(np.abs(z)) * 2
print(f'z = {z:.4f}, z_critical = {z_critical:.4f}, {"accept" if np.abs(z) <= z_critical else "reject"} null hypothesis')
print(f'z = {z:.4f}, p = {p:.4f}, {"accept" if p > alpha else "reject"} null hypothesis')

z = -0.61, z_critical = 1.96, accept null hypothesis
z = -0.61, p = 0.54, accept null hypothesis


##### Two sample z-test
- formula: $$z = \frac{(\bar{x}_1 - \bar{x}_2) - (\mu_1 - \mu_2)}{\sqrt{\frac{\sigma_1^2}{n_1} + \frac{\sigma_2^2}{n_2}}}$$
    - $\bar{x}_1, \bar{x}_2$: sample means
    - $\mu_1, \mu_2$: population means
    - $\sigma_1, \sigma_2$: population standard deviations
    - $n_1, n_2$: sample sizes

In [15]:
# two sample z-test, data from https://www.theopeneducator.com/doe/hypothesis-Testing-Inferential-Statistics-Analysis-of-Variance-ANOVA/Two-Sample-Z-Test
# height of US and Swedish males, 30 data points each
us = [69.12, 66.88, 74.82, 67.00, 69.12, 65.00, 71.00, 66.76, 72.12, 72.94, 69.18, 66.18, 64.94, 71.76, 70.12, 71.00, 71.88, 65.24, 70.06, 71.94, 72.12, 66.88, 73.82, 74.00, 71.18, 67.88, 65.94, 68.88, 68.00, 75.12]
swed = [74.56, 71.89, 73.00, 67.78, 72.22, 68.00, 73.56, 75.00, 68.22, 69.00, 68.00, 72.00, 73.56, 72.56, 75.00, 68.33, 71.67, 72.44, 75.00, 71.89, 72.00, 70.00, 69.22, 74.44, 68.00, 73.89, 70.00, 70.44, 70.22, 73.33]

n1, n2 = len(us), len(swed)
x1, x2 = np.mean(us), np.mean(swed)
sigma1, sigma2 = 3.12, 2.44
# the hypothesized difference is 0
s1, s2 = np.std(us, ddof=1), np.std(swed, ddof=1)
print(f'n1 = {n1}, n2 = {n2}, x1 = {x1:.4f}, x2 = {x2:.4f}, s1 = {s1:.4f}, s2 = {s2:.4f}, sigma1 = {sigma1:.4f}, sigma2 = {sigma2:.4f}')

z = ((x1 - x2) - (0)) / np.sqrt(sigma1**2/n1 + sigma2**2/n2)
alpha = 0.05
z_critical = stats.norm.ppf(1-alpha)
p = stats.norm.sf(np.abs(z)) * 2
print(f'z = {z:.4f}, z_critical = {z_critical:.4f}, {"accept" if np.abs(z) <= z_critical else "reject"} null hypothesis')
print(f'z = {z:.4f}, p = {p:.4f}, {"accept" if p > alpha else "reject"} null hypothesis')

n1 = 30, n2 = 30, x1 = 69.6960, x2 = 71.5073, s1 = 3.0171, s2 = 2.4070, sigma1 = 3.1200, sigma2 = 2.4400
z = -2.5048, z_critical = 1.6449, reject null hypothesis
z = -2.5048, p = 0.0123, reject null hypothesis


##### Proportion z-test
- assumptions:
    - sample size is large enough to assume normal distribution of sample proportion, check if $np \geq 5$ and $n(1-p) \geq 5$
- formula: $$z = \frac{p - p_0}{\sqrt{\frac{p_0(1-p_0)}{n}}}$$
    - $p$: sample proportion
    - $p_0$: population proportion
    - $n$: sample size

In [9]:
# proportion z-test, from https://www.theopeneducator.com/doe/hypothesis-Testing-Inferential-Statistics-Analysis-of-Variance-ANOVA/Population-Proportion-Test-Single-Sample
p0 = 0.057338
p_hat = 0.064
n = 328200000
assert n * p0 >= 5 and n * (1-p0) >= 5
z = (p_hat - p0) / np.sqrt(p0 * (1-p0) / n)
alpha = 0.05
z_critical = stats.norm.ppf(1-alpha/2)
p = stats.norm.sf(np.abs(z)) * 2
print(f'z = {z:.4f}, z_critical = {z_critical:.4f}, {"accept" if np.abs(z) <= z_critical else "reject"} null hypothesis')
print(f'z = {z:.4f}, p = {p:.4f}, {"accept" if p > alpha else "reject"} null hypothesis')

z = 519.13, z_critical = 1.96, reject null hypothesis
z = 519.13, p = 0.00, reject null hypothesis
